In [1]:
import requests
import pprint
import json
import re

with open('ip-logfile.log', 'r') as fo:
    log = fo.read()

ip_reg = re.compile(r";((\d{1,3}).(\d{1,3}).(\d{1,3}).(\d{1,3}));")
all_ip = set(ip_reg.findall(log))

api_url = 'https://www.whoisxmlapi.com/whoisserver/WhoisService?apiKey=at_4yWBY4ApasrNfmPuIBzZupJOYSSrf&outputFormat=JSON&domainName='

whois = {ip[0] : json.loads(requests.get('{}{}'.format(api_url, ip[0])).text) for ip in all_ip}

with open('whois.py', 'w') as fo:
    fo.write('whois={}'.format(whois))

In [5]:
import whois
import pprint

registrants = {}

for key in whois.whois.keys():
    if 'registrant' in whois.whois[key]['WhoisRecord']['registryData']:
        registry_data = whois.whois[key]['WhoisRecord']['registryData']
    else:
        registry_data = whois.whois[key]['WhoisRecord']['subRecords'][0]

    registrants[key] = {
        'ip' : key,
        'organization' : registry_data['registrant']['organization'],
        'netname' : registry_data['customField2Value'],
        'street' : registry_data['registrant']['street1'] if 'street1' in registry_data['registrant'] else 'None',
        'city' : registry_data['registrant']['city'] if 'city' in registry_data['registrant'] else 'None',
        'postalcode' : registry_data['registrant']['postalCode'] if 'postalCode' in registry_data['registrant'] else 'None',
        'country' : registry_data['registrant']['country'],
    }

pprint.pprint(registrants)

{'108.87.82.181': {'city': 'Redmond',
                   'country': 'UNITED STATES',
                   'ip': '108.87.82.181',
                   'netname': 'SBCIS-SBIS',
                   'organization': 'AT&T Corp.',
                   'postalcode': '98052',
                   'street': '16331 NE 72nd Way'},
 '139.99.172.11': {'city': 'SYDNEY',
                   'country': 'AU',
                   'ip': '139.99.172.11',
                   'netname': 'VPS-SYD',
                   'organization': 'OVH Australia PTY LTD',
                   'postalcode': '2060',
                   'street': '90 ARTHUR STREET  NORTH SYDNEY NSW'},
 '162.247.74.217': {'city': 'Brooklyn',
                    'country': 'UNITED STATES',
                    'ip': '162.247.74.217',
                    'netname': 'CALYX-INSTITUTE-V4-1',
                    'organization': 'The Calyx Institute',
                    'postalcode': '11232',
                    'street': '254 36th Street'},
 '174.130.215.196': {'c

In [8]:
import pymysql

cnx = pymysql.connect(user='dev', password='ax2', host='127.0.0.1', port=3306, db='test')
query = 'INSERT INTO whois (ip, city, country, netname, organization, postalcode, street) VALUES (%(ip)s, %(city)s, %(country)s, %(netname)s, %(organization)s, %(postalcode)s, %(street)s)'

with cnx.cursor() as cursor:
    for registrant in registrants.values():
        cursor.execute(query, registrant)

cnx.commit()
cnx.close()